# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np
document = ET.parse( './data/mondial_database.xml' )

In [6]:
df = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('infant_mortality'):
        country = element.find('name').text
        mortality = subelement.text
        df.append([country, float(mortality)])

In [7]:
df2 = pd.DataFrame(df, columns = ['country', 'mortality'])
df2.head()

,country,mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.90
3,Serbia,6.16
4,Andorra,3.69


In [8]:
df2.sort_values('mortality').head(10)

,country,mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [9]:
df = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        for subelement2 in subelement.getiterator('population'):
            country = element.find('name').text
            city = subelement.find('name').text
            year = int(subelement2.get('year'))
            measured = subelement2.get('measured')
            pop = int(subelement2.text)
            df.append([country, city, year, measured, pop])
df2 = pd.DataFrame(df, columns = ['country', 'city', 'year', 'measured', 'pop'])
recentpop = df2.sort_values(['city', 'year']).groupby('city').last().sort_values('pop', ascending = False)
recentpop.head(10)

,country,year,measured,pop
city,,,,
Shanghai,China,2010,census,22315474
Istanbul,Turkey,2012,admin.,13710512
Mumbai,India,2011,census,12442373
Moskva,Russia,2013,estimate,11979529
Beijing,China,2010,census,11716620
São Paulo,Brazil,2010,census,11152344
Tianjin,China,2010,census,11090314
Guangzhou,China,2010,census,11071424
Delhi,India,2011,census,11034555


In [215]:
df = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('ethnicgroup'):
        country = element.find('name').text
        ethnic = subelement.text
        percent = float(subelement.get('percentage'))
        df.append([country, ethnic, percent])
df2 = pd.DataFrame(df, columns = ['country', 'ethnicity', 'percentage'])
df2.head()

,country,ethnicity,percentage
0,Albania,Albanian,95.0
1,Albania,Greek,3.0
2,Greece,Greek,93.0
3,Macedonia,Macedonian,64.2
4,Macedonia,Albanian,25.2


In [216]:
recentpop2 = recentpop.reset_index()
merged = df2.merge(recentpop2, on = 'country')
merged.head()

,country,ethnicity,percentage,city,year,measured,pop
0,Albania,Albanian,95.0,Tirana,2011,census,418495
1,Albania,Albanian,95.0,Durrës,2011,census,113249
2,Albania,Albanian,95.0,Vlorë,2011,census,79513
3,Albania,Albanian,95.0,Elbasan,2011,census,78703
4,Albania,Albanian,95.0,Shkodër,2011,census,77075


In [217]:
countrypop = merged[['country', 'pop']].groupby(['country']).sum()
countrypop = pd.DataFrame(countrypop).reset_index()
countrypop.head()

,country,pop
0,Afghanistan,13484400
1,Albania,1636374
2,Algeria,14161380
3,Andorra,111280
4,Angola,22052000


In [218]:
merged2 = merged.merge(countrypop, on = 'country')
merged2 = merged2.groupby('ethnicity').first()
merged2.head()

,country,percentage,city,year,measured,pop_x,pop_y
ethnicity,,,,,,,
Acholi,Uganda,4.0,Kampala,2014,prelim. census,1516210,22478628
Afar,Djibouti,35.0,Djibouti,2009,census,475322,1425966
African,Andorra,5.0,Andorra la Vella,2011,admin.,22256,111280
African-white-Indian,Martinique,90.0,Fort-de-France,2009,None,100000,200000
Afro-Asian,Saudi Arabia,10.0,Riyadh,2004,census,4087152,20734444


In [219]:
type(merged2.percentage[0])

numpy.float64

In [241]:
merged2['ethnicity_pop'] = merged2['percentage']*.01*merged2['pop_y']
merged2['ethnicity_pop'] = round(merged2['ethnicity_pop'], 0)
merged2[['measured', 'ethnicity_pop']].sort_values('ethnicity_pop', ascending = False).head(10)

,measured,ethnicity_pop
ethnicity,,
Han Chinese,census,292861924.0
Indo-Aryan,census,277328647.0
Persian,census,132164240.0
English,census,99496974.0
Dravidian,census,96294669.0
Viet/Kinh,census,86667439.0
Javanese,census,83929439.0
Azerbaijani,census,62194936.0
Japanese,estimate,47780296.0


In [86]:
df = []
for element in document.iterfind('river'):
    river = element.find('name').text
    try:
        length = float(element.find('length').text)
    except:
        length = None
    country = element.get('country')
    df.append([river, length, country])
df2 = pd.DataFrame(df, columns = ['river', 'length', 'country'])
df2.sort_values('length', ascending = False).head(1)

,river,length,country
174,Amazonas,6448.0,CO BR PE


In [87]:
df = []
for element in document.iterfind('lake'):
    lake = element.find('name').text
    try:
        area = float(element.find('area').text)
    except:
        area = None
    country = element.get('country')
    df.append([lake, area, country])
df2 = pd.DataFrame(df, columns = ['name', 'area', 'country'])
df2.sort_values('area', ascending = False).head(1)

,name,area,country
54,Caspian Sea,386400.0,R AZ KAZ IR TM


In [92]:
df = []
for element in document.iterfind('airport'):
    airport = element.find('name').text
    try:
        elevation = float(element.find('elevation').text)
    except:
        elevation = None
    country = element.get('country')
    df.append([airport, elevation, country])
df2 = pd.DataFrame(df, columns = ['name', 'elevation', 'country'])
df2.sort_values('elevation', ascending = False).head(1)

,name,elevation,country
80,El Alto Intl,4063.0,BOL
